In [1]:
!sudo pip3 install pygame -q 

The directory '/home/finn/.cache/pip/http' or its parent directory is not owned by the current user and the cache has been disabled. Please check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.
The directory '/home/finn/.cache/pip' or its parent directory is not owned by the current user and caching wheels has been disabled. check the permissions and owner of that directory. If executing pip with sudo, you may want sudo's -H flag.


In [36]:
import snakai
import agents
import numpy as np
import math
import random
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple
from itertools import count
from copy import deepcopy

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision.transforms as T
import datetime

cpu = torch.device("cpu")
if torch.cuda.is_available():
    device = torch.device("cuda:1")
else:
    device = cpu

## Define how to play game and replay memory

In [2]:
game_size = (10,10)
def tuple_to_torch(tup):
    return torch.from_numpy(np.array(tup))

action2ind = {'right' : 0,
             'left' : 1,
             'up' : 2,
             'down' : 3}
ind2action = {val: key for key, val in action2ind.items()}


In [3]:
def play_game(snake, agent, epsilon = 0.05):
    cum_reward = 0.0
    snake.on_init()
    state, reward, ended = snake.on_feedback()

    for i in range(200):
        action = agent(state, th = epsilon)
        next_state, reward, ended = snake.step(action)
        cum_reward += float(reward)
        
        # Keep all the games:
        memory.push(state, action, next_state, reward, ended)
        state = next_state
        if ended == 1:
            return cum_reward, i
    return cum_reward, i


In [4]:
Transition = namedtuple('Transition',
                        ('state', 'action', 'next_state', 'reward','ended'))

class ReplayMemory(object):

    def __init__(self, capacity):
        self.capacity = capacity
        self.memory = []
        self.position = 0

    def push(self, *args):
        """Saves a transition."""
        if len(self.memory) < self.capacity:
            self.memory.append(None)
        self.memory[self.position] = Transition(*args)
        self.position = (self.position + 1) % self.capacity

    def sample(self, batch_size):
        return random.sample(self.memory, batch_size)

    def __len__(self):
        return len(self.memory)
    
memory = ReplayMemory(5000)

## Define agent

In [34]:
ch = 64
ksize = 4
class DQN(nn.Module):

    def __init__(self):
        super(DQN, self).__init__()
        self.conv1 = nn.Conv2d(3, ch, kernel_size=ksize, stride=2, padding = 2)
        self.conv2 = nn.Conv2d(ch, ch, kernel_size=ksize, stride=1, padding = 1)
        self.conv3 = nn.Conv2d(ch, ch, kernel_size=ksize, stride=1, padding = 0)
        #self.dense1 = nn.Linear(2592, 1024)
        self.head = nn.Linear(256, 4)

    def forward(self, x):
        #print(x.shape)
        x = F.relu(self.conv1(x))
        #print(x.shape)
        x = F.relu(self.conv2(x))
        #print(x.shape)
        x = F.relu(self.conv3(x))
        #print(x.shape)
        x = x.view(x.size(0), -1)
        # x = F.relu(self.dense1(x))
        return 2*F.tanh(self.head(x))
    
model = DQN().to(device)
batch_size = 32
optimizer = optim.Adam(model.parameters(), lr = 0.001) # , weight_decay = 0.001

In [40]:
reward_hat.max(1)[0]

tensor(1.00000e-02 *
       [ 8.9083,  8.5205,  8.2099,  9.3867,  9.0999,  8.3634,  9.2137,
         7.7036,  7.5422,  7.8198,  7.8792,  8.5759,  9.0187,  8.1598,
         7.6688,  7.9860,  9.4241,  8.0003,  8.7647,  8.3466,  9.0797,
         9.2634,  7.9579,  8.5084,  7.3906,  8.2424,  8.4048,  8.8836,
         7.7570], device='cuda:1')

In [41]:
    # GET SAMPLE OF DATA
    transitions = memory.sample(batch_size)
    batch = Transition(*zip(*transitions))
    state_batch = tuple_to_torch(batch.state).float().to(device)
    next_state_batch = tuple_to_torch(batch.next_state).float().to(device)
    action_batch = tuple_to_torch(list(action2ind[a] for a in batch.action)).to(device)
    reward_batch = tuple_to_torch(batch.reward).float().to(device)


    ## Calculate expected reward:
    GAMMA = 0.99
    with torch.set_grad_enabled(False):
        not_ended_batch = 1 -torch.ByteTensor(batch.ended)
        next_states_non_final = next_state_batch[not_ended_batch]
        next_state_values = torch.zeros(batch_size).to(device)
        reward_hat = model(next_states_non_final)
        next_state_values[not_ended_batch] = reward_hat.max(1)[0]
        expected_state_action_values = next_state_values*GAMMA + reward_batch


    # Predict value function:
    yhat = model(state_batch)
    state_action_values = yhat.gather(1, action_batch.unsqueeze(1)).squeeze()

    loss = F.smooth_l1_loss(state_action_values, expected_state_action_values)
    optimizer.zero_grad()
    loss.backward()
    for param in model.parameters():
        param.data.clamp_(-1, 1)
    optimizer.step()


In [42]:
def train_batch():
    if len(memory) < batch_size:
        return 0
    
    # GET SAMPLE OF DATA
    transitions = memory.sample(batch_size)
    batch = Transition(*zip(*transitions))
    state_batch = tuple_to_torch(batch.state).float().to(device)
    next_state_batch = tuple_to_torch(batch.next_state).float().to(device)
    action_batch = tuple_to_torch(list(action2ind[a] for a in batch.action)).to(device)
    reward_batch = tuple_to_torch(batch.reward).float().to(device)


    ## Calculate expected reward:
    GAMMA = 0.99
    with torch.set_grad_enabled(False):
        not_ended_batch = 1 -torch.ByteTensor(batch.ended)
        next_states_non_final = next_state_batch[not_ended_batch]
        next_state_values = torch.zeros(batch_size).to(device)
        reward_hat = model(next_states_non_final)
        next_state_values[not_ended_batch] = reward_hat.max(1)[0]
        expected_state_action_values = next_state_values*GAMMA + reward_batch


    # Predict value function:
    yhat = model(state_batch)
    state_action_values = yhat.gather(1, action_batch.unsqueeze(1)).squeeze()

    loss = F.smooth_l1_loss(state_action_values, expected_state_action_values)
    optimizer.zero_grad()
    loss.backward()
    for param in model.parameters():
        param.data.clamp_(-1, 1)
    optimizer.step()

    return loss.data

In [107]:
from agents import random_legal_move
def deep_agent(state, th):
    
    if random.random() < th:
        return random_legal_move(state)
    
    state = torch.unsqueeze(torch.from_numpy(state),0).float().to(device)
    yhat = model(state)
    action = [ind2action[a] for a in yhat.argmax(1).data.cpu().numpy()]
    if len(action) > 1:
        raise Exception
    action = action[0]
    return action

In [109]:
snake = snakai.Snake(render=False, 
                     game_size = game_size, 
                     time_reward = -0.01)

# Warmup memory:
for _ in range(10):
    play_game(snake, deep_agent, epsilon = 1)

In [110]:
def evaluate_agent(n = 100, epsilon = 0.05):
    rewards = np.zeros(n)
    for ep in range(n):
        rewards[ep],i = play_game(snake, deep_agent, epsilon = epsilon)
        
    return np.mean(rewards)

def save_checkpoint():
    filename = "models/snake_legalexp_ep:%02d-reward:%.2f.pth" %( ep, eval_reward)
    torch.save(model.state_dict(), filename)
    return True

In [ ]:
REPORT_INTERVAL = 100
EVAL_INTERVAL = 2000
R = []
L = []
play_length = []

EPS_START = 0.9
EPS_END = 0.01
decay = 0.1/2000
start_ep = 0

for ep in range(start_ep,1000000):
    
    # Play one game:
    epsilon = max(EPS_START - decay*(ep), EPS_END)
    r, i = play_game(snake, deep_agent, epsilon = epsilon)
    R.append(r)
    play_length.append(i)
    
    # Train:
    for _ in range(i):
        l = train_batch()
        L.append(float(l))
    
    if ep % REPORT_INTERVAL == 0:
        print("%s: ep: %s \t reward: %.3f \t loss: %.4f \t game len: %.1f \t epsilon: %.2f" % 
              (str(datetime.datetime.now()), ep, np.mean(R), np.mean(L), np.mean(play_length), epsilon))
        R = []
        L = []
        play_length = []
    
    if ep % EVAL_INTERVAL == 0:
        eval_reward = evaluate_agent()
        save_checkpoint()
        print("%s: ep: %s \t Reward evaluation: %.2f" % (str(datetime.datetime.now()), ep, eval_reward))

2018-05-28 13:38:58.261089: ep: 0 	 reward: -1.430 	 loss: 0.0585 	 game len: 43.0 	 epsilon: 0.90
2018-05-28 13:38:58.565142: ep: 0 	 Reward evaluation: -0.94
2018-05-28 13:39:11.462230: ep: 100 	 reward: -0.867 	 loss: 0.0536 	 game len: 32.1 	 epsilon: 0.90
2018-05-28 13:39:38.315400: ep: 200 	 reward: -0.593 	 loss: 0.0432 	 game len: 63.2 	 epsilon: 0.89
2018-05-28 13:40:06.998426: ep: 300 	 reward: -0.779 	 loss: 0.0284 	 game len: 73.7 	 epsilon: 0.89
2018-05-28 13:40:36.635899: ep: 400 	 reward: -0.470 	 loss: 0.0232 	 game len: 78.2 	 epsilon: 0.88
2018-05-28 13:41:08.635133: ep: 500 	 reward: -0.601 	 loss: 0.0229 	 game len: 81.2 	 epsilon: 0.88
2018-05-28 13:41:39.795866: ep: 600 	 reward: -0.181 	 loss: 0.0245 	 game len: 79.6 	 epsilon: 0.87
2018-05-28 13:42:16.344601: ep: 700 	 reward: -0.371 	 loss: 0.0195 	 game len: 93.5 	 epsilon: 0.86
2018-05-28 13:43:03.788676: ep: 800 	 reward: -0.338 	 loss: 0.0135 	 game len: 123.2 	 epsilon: 0.86
2018-05-28 13:43:52.668336: ep:

In [ ]:
# Training time now:
ep

In [ ]:
# evaluate agent with 5% epsilon greedy policy:
evaluate_agent(n = 1000, epsilon = 0.05)

In [ ]:
# Evaluate agent with greedy policy:
evaluate_agent(n = 1000, epsilon = 0.0)

In [ ]:
snake = snakai.Snake(render=False, 
                     game_size = game_size, 
                     time_reward = -0.01)
snake.on_init()
state, reward, done = snake.on_feedback()

for _ in range(10):
    print(play_game(snake, deep_agent, epsilon = 0.0))